In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
import sys
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
sys.path.append("../../src/")
import data_utils

# Dataset Utilities

In [5]:
peng_dir = dict(
    lap14 = "../../data/absa/en/peng/14lap",
    res14 = "../../data/absa/en/peng/14res",
    res15 = "../../data/absa/en/peng/15res",
    res16 = "../../data/absa/en/peng/16res"
)

wan_dir = dict(
    res15 = "../../data/absa/en/wan/interim/rest15",
    res16 = "../../data/absa/en/wan/interim/rest16"
)
    
zhang_dir = dict(
    res15 = "../../data/absa/en/zhang/interim/interim_2/rest15",
    res16 = "../../data/absa/en/zhang/interim/interim_2/rest16"
)

william_dir = dict(
    hotel = "../../data/absa/id/william"
)

peng = dict(
    lap14 = dict(
        train = data_utils.read_data(path=peng_dir["lap14"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["lap14"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["lap14"] + "/test_triplets.txt",
                                     target_format="aos")
    ),
    res14 = dict(
        train = data_utils.read_data(path=peng_dir["res14"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["res14"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["res14"] + "/test_triplets.txt",
                                     target_format="aos")
    ),
    res15 = dict(
        train = data_utils.read_data(path=peng_dir["res15"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["res15"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["res15"] + "/test_triplets.txt",
                                     target_format="aos")
    ),
    res16 = dict(
        train = data_utils.read_data(path=peng_dir["res16"] + "/train_triplets.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=peng_dir["res16"] + "/dev_triplets.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=peng_dir["res16"] + "/test_triplets.txt",
                                     target_format="aos")
    )
)

wan = dict(
    res15 = dict(
        train = data_utils.read_data(path=wan_dir["res15"] + "/train.txt",
                                     target_format="acs"),
        val = data_utils.read_data(path=wan_dir["res15"] + "/dev.txt",
                                     target_format="acs"),
        test = data_utils.read_data(path=wan_dir["res15"] + "/test.txt",
                                     target_format="acs")
    ),
    res16 = dict(
        train = data_utils.read_data(path=wan_dir["res16"] + "/train.txt",
                                     target_format="acs"),
        val = data_utils.read_data(path=wan_dir["res16"] + "/dev.txt",
                                     target_format="acs"),
        test = data_utils.read_data(path=wan_dir["res16"] + "/test.txt",
                                     target_format="acs")
    )
)

zhang = dict(
    res15 = dict(
        train = data_utils.read_data(path=zhang_dir["res15"] + "/train.txt",
                                     target_format="acso"),
        val = data_utils.read_data(path=zhang_dir["res15"] + "/dev.txt",
                                     target_format="acso"),
        test = data_utils.read_data(path=zhang_dir["res15"] + "/test.txt",
                                     target_format="acso")
    ),
    res16 = dict(
        train = data_utils.read_data(path=zhang_dir["res16"] + "/train.txt",
                                     target_format="acso"),
        val = data_utils.read_data(path=zhang_dir["res16"] + "/dev.txt",
                                     target_format="acso"),
        test = data_utils.read_data(path=zhang_dir["res16"] + "/test.txt",
                                     target_format="acso")
    )
)

william = dict(
    hotel = dict(
        train = data_utils.read_data(path=william_dir["hotel"] + "/train.txt",
                                     target_format="aos"),
        val = data_utils.read_data(path=william_dir["hotel"] + "/dev.txt",
                                     target_format="aos"),
        test = data_utils.read_data(path=william_dir["hotel"] + "/test.txt",
                                     target_format="aos")
    )
)

# Data Preprocessing 1

In [6]:
data_utils.SENTIMENT_ELEMENT = {'a' : "aspect", 'o' : "opinion", 's' : "sentiment", 'c' : "category"}

1. AOS (ASTE)
    * AO
    * AS
    * A
    * O

2. ACS (TASD)
    * AS
    * CS
    * A
    * C

3. ACOS
    * AO
    * AS
    * CS
    * A
    * O
    * C

In [7]:
task_tree = {
    "oas" : ["oa","as",'a','o'],
    "asc" : ["as","sc",'a','c'],
    "oasc" : ["oa","as","sc",'a','o','c']
}

all_task = []
for k,v1 in task_tree.items():
    if k not in all_task:
        all_task.append(k)
    for v2 in v1:
        if v2 not in all_task:
            all_task.append(v2)

print(all_task)

['oas', 'oa', 'as', 'a', 'o', 'asc', 'sc', 'c', 'oasc']


In [8]:
data_utils.remove_duplicate_targets(data_utils.reduce_targets([{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "positive"},{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "negative"}],"ao"))

[{'aspect': 'battery life', 'opinion': 'good'}]

Handle mix may not be a must, but we'll see it later. Will be problematic if like as (UABSA / E2E ABSA) used for training AOS (ASTE) --> may be for further experiment because we will insert imputing later on

In [9]:
data_utils.handle_mix_sentiment(data_utils.reduce_targets([{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "positive"},{'aspect': 'battery life', 'opinion': 'good', "sentiment" : "negative"}],"aos"))

[{'aspect': 'battery life', 'opinion': 'good', 'sentiment': 'mixed'}]

In [10]:
from copy import deepcopy

# Peng (ASTE/AOS)
peng_intermediate = dict()

for domain, v1 in peng.items():
    peng_intermediate[domain] = dict()
    for task in ["oas"] + task_tree["oas"]:
        peng_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = peng[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            peng_intermediate[domain][task][split] = ds_copy

In [11]:
# Wan (TASD/ACS)
wan_intermediate = dict()

for domain, v1 in wan.items():
    wan_intermediate[domain] = dict()
    for task in ["asc"] + task_tree["asc"]:
        wan_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = wan[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            wan_intermediate[domain][task][split] = ds_copy

In [12]:
# Zhang (ACOS)
zhang_intermediate = dict()

for domain, v1 in zhang.items():
    zhang_intermediate[domain] = dict()
    for task in ["oasc"] + task_tree["oasc"]:
        zhang_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = zhang[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            zhang_intermediate[domain][task][split] = ds_copy

In [13]:
# William (AOS ID)
william_intermediate = dict()

for domain, v1 in william.items():
    william_intermediate[domain] = dict()
    for task in ["oas"] + task_tree["oas"]:
        william_intermediate[domain][task] = dict()
        for split in v1.keys():
            ds = william[domain][split]
            ds_copy = deepcopy(ds)
            for i in range(len(ds_copy)):
                # Reduce
                ds_copy[i]["target"] = data_utils.reduce_targets(ds_copy[i]["target"],task)
                # Remove Duplicates
                ds_copy[i]["target"] = data_utils.remove_duplicate_targets(ds_copy[i]["target"])
            william_intermediate[domain][task][split] = ds_copy

# Answer Engineering

In [14]:
mask = "<extra_id_X>"

In [15]:
def construct_answer(targets,se_order):
    result = []
    counter = 0
    for t in targets:
        constructed_t = ""
        for se in se_order:
            if counter > 99:
                raise Exception("Extra id more than 99!")
            constructed_t += ' ' + mask.replace('X',str(counter)) + ' ' + t[data_utils.SENTIMENT_ELEMENT[se]]
            counter += 1
        constructed_t = constructed_t.strip()
        result.append(constructed_t)
    result = " ; ".join(result)
    return result

In [16]:
construct_answer(peng_intermediate["lap14"]["oas"]["train"][4]["target"],"oas")

'<extra_id_0> no <extra_id_1> GUI <extra_id_2> negative ; <extra_id_3> dark <extra_id_4> screen <extra_id_5> negative ; <extra_id_6> steady <extra_id_7> power light <extra_id_8> neutral ; <extra_id_9> steady <extra_id_10> hard drive light <extra_id_11> negative'

# Prompt Engineering

In [17]:
def construct_prompt(text,se_order):
    prompt = []
    for counter, se in enumerate(se_order):
        prompt.append(data_utils.SENTIMENT_ELEMENT[se] + " : " + mask.replace('X',str(counter)))
    prompt = " ,".join(prompt)
    result = text + "| " + prompt
    return result

In [18]:
construct_prompt(peng_intermediate["lap14"]["oas"]["train"][4]["text"],"oas")

'One night I turned the freaking thing off after using it , the next day I turn it on , no GUI , screen all dark , power light steady , hard drive light steady and not flashing as it usually does .| opinion : <extra_id_0> ,aspect : <extra_id_1> ,sentiment : <extra_id_2>'

# Answer Catch

In [19]:
import re

def catch_answer(output,se_order):
    output = output.replace("<pad>",'')
    output = output.replace("</s>",'')
    pattern = r""
    for se in se_order:
        if se != 's':
            pattern += f"<extra_id_\d+>\s*(?P<{data_utils.SENTIMENT_ELEMENT[se]}>[^;]+)\s*"
        else:
            pattern += f"<extra_id_\d+>\s*(?P<{data_utils.SENTIMENT_ELEMENT['s']}>positive|negative|neutral)\s*"
    found = [found_iter.groupdict() for found_iter in re.finditer(pattern,output)]
    for i in range(len(found)):
        for k, v in found[i].items():
            found[i][k] = found[i][k].strip()
    return found

In [20]:
output = construct_answer(peng_intermediate["lap14"]["oas"]["train"][4]["target"],"oas")
se_order = "oas"
catch_answer(output,se_order)

[{'opinion': 'no', 'aspect': 'GUI', 'sentiment': 'negative'},
 {'opinion': 'dark', 'aspect': 'screen', 'sentiment': 'negative'},
 {'opinion': 'steady', 'aspect': 'power light', 'sentiment': 'neutral'},
 {'opinion': 'steady', 'aspect': 'hard drive light', 'sentiment': 'negative'}]

In [21]:
output

'<extra_id_0> no <extra_id_1> GUI <extra_id_2> negative ; <extra_id_3> dark <extra_id_4> screen <extra_id_5> negative ; <extra_id_6> steady <extra_id_7> power light <extra_id_8> neutral ; <extra_id_9> steady <extra_id_10> hard drive light <extra_id_11> negative'

# Data Preprocessing 2

In [22]:
from datasets import Dataset

peng_2 = dict()
for domain, v1 in peng_intermediate.items():
    peng_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["oas"]:
        for el in peng_intermediate[domain][basic_task]["train"]:
            peng_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in peng_intermediate[domain]["oas"]["val"]:
        peng_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    # TEST
    for el in peng_intermediate[domain]["oas"]["test"]:
        peng_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    peng_2[domain]["train"] = Dataset.from_list(peng_2[domain]["train"])
    peng_2[domain]["val"] = Dataset.from_list(peng_2[domain]["val"])
    peng_2[domain]["test"] = Dataset.from_list(peng_2[domain]["test"])

wan_2 = dict()
for domain, v1 in wan_intermediate.items():
    wan_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["asc"]:
        for el in wan_intermediate[domain][basic_task]["train"]:
            wan_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in wan_intermediate[domain]["asc"]["val"]:
        wan_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"asc"),
                "output" : construct_answer(el["target"],"asc"),
                "task" : "asc"
            })
    # TEST
    for el in wan_intermediate[domain]["asc"]["test"]:
        wan_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"asc"),
                "output" : construct_answer(el["target"],"asc"),
                "task" : "asc"
            })
    wan_2[domain]["train"] = Dataset.from_list(wan_2[domain]["train"])
    wan_2[domain]["val"] = Dataset.from_list(wan_2[domain]["val"])
    wan_2[domain]["test"] = Dataset.from_list(wan_2[domain]["test"])

zhang_2 = dict()
for domain, v1 in zhang_intermediate.items():
    zhang_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["oasc"]:
        for el in zhang_intermediate[domain][basic_task]["train"]:
            zhang_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in zhang_intermediate[domain]["oasc"]["val"]:
        zhang_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"oasc"),
                "output" : construct_answer(el["target"],"oasc"),
                "task" : "oasc"
            })
    # TEST
    for el in zhang_intermediate[domain]["oasc"]["test"]:
        zhang_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"oasc"),
                "output" : construct_answer(el["target"],"oasc"),
                "task" : "oasc"
            })
    zhang_2[domain]["train"] = Dataset.from_list(zhang_2[domain]["train"])
    zhang_2[domain]["val"] = Dataset.from_list(zhang_2[domain]["val"])
    zhang_2[domain]["test"] = Dataset.from_list(zhang_2[domain]["test"])

william_2 = dict()
for domain, v1 in william_intermediate.items():
    william_2[domain] = {
        "train" : [], # basic task
        "val" : [], # complex task
        "test" : [] # complex task
    }
    # TRAIN
    for basic_task in task_tree["oas"]:
        for el in william_intermediate[domain][basic_task]["train"]:
            william_2[domain]["train"].append({
                    "input" : construct_prompt(el["text"],basic_task),
                    "output" : construct_answer(el["target"],basic_task),
                    "task" : basic_task
                })
    # VAL
    for el in william_intermediate[domain]["oas"]["val"]:
        william_2[domain]["val"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    # TEST
    for el in william_intermediate[domain]["oas"]["test"]:
        william_2[domain]["test"].append({
                "input" : construct_prompt(el["text"],"oas"),
                "output" : construct_answer(el["target"],"oas"),
                "task" : "oas"
            })
    william_2[domain]["train"] = Dataset.from_list(william_2[domain]["train"])
    william_2[domain]["val"] = Dataset.from_list(william_2[domain]["val"])
    william_2[domain]["test"] = Dataset.from_list(william_2[domain]["test"])

In [23]:
william_2["hotel"]["train"][69]

{'input': 'tempat yag bagus dan nyaman untuk istirahat tetapi tolong tvnya perlu di perbaiki channelnya karena banyak semutnya digambar dan water heaternya tidak bisa jadi mandi air dingin terus .| opinion : <extra_id_0> ,aspect : <extra_id_1>',
 'output': '<extra_id_0> bagus <extra_id_1> tempat ; <extra_id_2> nyaman <extra_id_3> tempat ; <extra_id_4> perlu di perbaiki <extra_id_5> tvnya ; <extra_id_6> tidak bisa <extra_id_7> water heaternya',
 'task': 'oa'}

# Prepare Tokenized Dataset

## English

In [24]:
tokenizer_en = AutoTokenizer.from_pretrained("t5-base")

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [25]:
encoding_args = {
    "max_length" : 512,
    "padding" : True,
    "truncation" : True,
    "return_tensors" : "pt"
}

In [26]:
def encode_en(dataset):
    result = tokenizer_en(dataset["input"], text_target=dataset["output"], **encoding_args)
    return result

In [27]:
peng_tok = dict()
for domain, v1 in peng_2.items():
    peng_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            peng_tok[domain][split] = peng_2[domain][split].map(encode_en,batched=True,remove_columns=["input","output","task"])
        else:
            peng_tok[domain][split] = encode_en(peng_2[domain][split])

In [28]:
wan_tok = dict()
for domain, v1 in wan_2.items():
    wan_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            wan_tok[domain][split] = wan_2[domain][split].map(encode_en,batched=True,remove_columns=["input","output","task"])
        else:
            wan_tok[domain][split] = encode_en(wan_2[domain][split])

In [29]:
zhang_tok = dict()
for domain, v1 in zhang_2.items():
    zhang_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            zhang_tok[domain][split] = zhang_2[domain][split].map(encode_en,batched=True,remove_columns=["input","output","task"])
        else:
            zhang_tok[domain][split] = encode_en(zhang_2[domain][split])

## Indo

In [30]:
tokenizer_id = AutoTokenizer.from_pretrained("Wikidepia/IndoT5-base")

In [31]:
def encode_id(dataset):
    result = tokenizer_id(dataset["input"], text_target=dataset["output"], **encoding_args)
    return result

In [32]:
william_tok = dict()
for domain, v1 in william_2.items():
    william_tok[domain] = dict()
    for split, v2 in v1.items():
        if split != "test":
            william_tok[domain][split] = william_2[domain][split].map(encode_id,batched=True,remove_columns=["input","output","task"])
        else:
            william_tok[domain][split] = encode_id(william_2[domain][split])

# Data Collator

## English

In [33]:
from transformers import DataCollatorForSeq2Seq

data_collator_en = DataCollatorForSeq2Seq(tokenizer=tokenizer_en)

## Indo

In [34]:
data_collator_id = DataCollatorForSeq2Seq(tokenizer=tokenizer_id)

# Compute Metrics

In [35]:
from transformers import EvalPrediction
from evaluation import recall, precision, f1_score, summary_score
from typing import List, Dict, Tuple
import numpy as np

def seperate_target_prediction_per_task(predictions:List[List[Dict]],targets:List[List[Dict]],tasks:List) -> Tuple[Dict[str,List],Dict[str,List]]:
    per_task_targets = {}
    per_task_predictions = {}
    for target, prediction, task in zip(targets,predictions,tasks):
        if task not in per_task_targets.keys():
            per_task_targets[task] = []
        if task not in per_task_predictions.keys():
            per_task_predictions[task] = []
        per_task_targets[task].append(target)
        per_task_predictions[task].append(prediction)
    return per_task_targets, per_task_predictions

def preprocess_eval_preds(eval_preds:EvalPrediction,decoding_args:Dict[str,str],tokenizer:AutoTokenizer):
    input_ids = eval_preds.inputs
    target_ids = eval_preds.label_ids
    pred_ids = eval_preds.predictions

    # In case the model returns more than the prediction logits
    if isinstance(input_ids, tuple):
        input_ids = input_ids[0]
    if isinstance(target_ids, tuple):
        target_ids = target_ids[0]
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    input_ids = np.argmax(input_ids,axis=-1) if len(input_ids.shape) == 3 else input_ids # in case not predict with generate
    target_ids = np.argmax(target_ids,axis=-1) if len(target_ids.shape) == 3 else target_ids # in case not predict with generate
    prediction_ids = np.argmax(pred_ids,axis=-1) if len(pred_ids.shape) == 3 else pred_ids # in case not predict with generate

    input_ids = [[token for token in row if token != -100] for row in input_ids]
    target_ids = [[token for token in row if token != -100] for row in target_ids]
    prediction_ids = [[token for token in row if token != -100] for row in prediction_ids]

    inputs = tokenizer.batch_decode(input_ids,**decoding_args)
    targets = tokenizer.batch_decode(target_ids,**decoding_args)
    predictions = tokenizer.batch_decode(prediction_ids,**decoding_args)

    return inputs, targets, predictions

def compute_metrics(eval_preds:EvalPrediction,decoding_args:Dict[str,str],tokenizer:AutoTokenizer,tasks:List) -> Dict[str,float]: # MAY NOT BE SUFFICIATE FOR CAUSAL LM
        """
        ### DESC
            Method to compute the metrics.
        ### PARAMS
        * eval_preds: EvalPrediction instance from training.
        * decoding_args: Decoding arguments.
        ### RETURN
        * metrics: Dictionary of metrics.
        """
        inputs, targets, predictions = preprocess_eval_preds(eval_preds,decoding_args,tokenizer)

        targets = [catch_answer(text,task) for text,task in zip(targets,tasks) if task != "non_absa"]
        predictions = [catch_answer(text,task) for text,task in zip(predictions,tasks) if task != "non_absa"]


        per_task_targets, per_task_predictions = seperate_target_prediction_per_task(predictions, targets, tasks)
        
        metrics = {}

        metrics["overall_recall"] = recall(predictions,targets)
        metrics["overall_precision"] = precision(predictions,targets)
        metrics["overall_f1_score"] = f1_score(predictions,targets)

        for task in per_task_targets.keys():
            if task == "non_absa":
                continue
            metrics[f"{task}_recall"] = recall(per_task_predictions[task],per_task_targets[task])
            metrics[f"{task}_precision"] = precision(per_task_predictions[task],per_task_targets[task])
            metrics[f"{task}_f1_score"] = f1_score(per_task_predictions[task],per_task_targets[task])
        
        return metrics

# Train Arguments

In [36]:
from transformers import Seq2SeqTrainingArguments

train_args = {
    "num_train_epochs": 20,
    "learning_rate": 3e-4,
    "save_total_limit": 2,
    "gradient_accumulation_steps": 1,
    "per_device_train_batch_size": 32,
    "per_device_eval_batch_size": 32,
    "save_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "logging_strategy" : "epoch",
    "metric_for_best_model": "overall_f1_score",
    "load_best_model_at_end": True,
    "adam_epsilon": 1e-08,
    "output_dir": "./t5",
    "logging_dir" : "./t5/log",
    "include_inputs_for_metrics" : True
}

train_args = Seq2SeqTrainingArguments(**train_args)

# Model

## English

In [37]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

## Indo

In [38]:
model_id = AutoModelForSeq2SeqLM.from_pretrained("Wikidepia/IndoT5-base")

# Train

In [39]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [40]:
from transformers import Seq2SeqTrainer

trainer = {
    "peng" : {},
    "wan" : {},
    "zhang" : {},
    "william" : {}
}

decoding_args = {
    "skip_special_tokens" : False
}

def preprocess_logits_for_metrics(logits, labels):
    pred_logits = logits[0] if isinstance(logits,tuple) else logits
    pred_ids = torch.argmax(pred_logits, dim=-1)
    return pred_ids, labels

In [41]:
from tqdm import tqdm

def generate_predictions(model,tokenizer,tokenized:torch.Tensor,device:torch.device=torch.device("cpu"),batch_size:int=16,max_len:int=512,decoding_args:Dict={}) -> List[str]:
    # Data loader
    input_ids_data_loader = torch.utils.data.DataLoader(tokenized["input_ids"],
                        batch_size=batch_size,shuffle=False)
    attention_mask_data_loader = torch.utils.data.DataLoader(tokenized["attention_mask"],
                        batch_size=batch_size,shuffle=False)
    # Predict
    model = model
    tokenizer = tokenizer
    tensor_predictions = []
    with torch.no_grad():
        for input_ids, attention_mask in tqdm(zip(input_ids_data_loader,attention_mask_data_loader)):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            tensor_predictions.extend(model.generate(input_ids=input_ids,attention_mask=attention_mask,max_length=max_len,pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id).cpu())
            input_ids = input_ids.cpu()
            attention_mask = attention_mask.cpu()
    tensor_predictions = [[token for token in row if token != -100] for row in tensor_predictions]
    predictions = tokenizer.batch_decode(tensor_predictions,**decoding_args)
    return predictions

# Peng Laptop 2014

In [42]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["peng"]["lap14"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["lap14"]["train"],
        eval_dataset = peng_tok["lap14"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["lap14"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["peng"]["lap14"].train()

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3624
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2280
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Overall Recall,Overall Precision,Overall F1 Score,Oas Recall,Oas Precision,Oas F1 Score
1,0.530700,0.164901,0.321739,0.452000,0.375905,0.321739,0.452000,0.375905


***** Running Evaluation *****
  Num examples = 219
  Batch size = 32
  Num examples = 219
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-114
Configuration saved in ./t5/checkpoint-114/config.json
Saving model checkpoint to ./t5/checkpoint-114
Configuration saved in ./t5/checkpoint-114/config.json
Model weights saved in ./t5/checkpoint-114/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-114/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-114/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-114/spiece.model
***** Running Evaluation *****
  Num examples = 219
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-228
Configuration saved in ./t5/checkpoint-228/config.json
Model weights saved in ./t5/checkpoint-228/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-228/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-228/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-228/spiece.mod

TrainOutput(global_step=2280, training_loss=0.03503607097817095, metrics={'train_runtime': 901.4619, 'train_samples_per_second': 80.403, 'train_steps_per_second': 2.529, 'total_flos': 8965382543769600.0, 'train_loss': 0.03503607097817095, 'epoch': 20.0})

In [43]:
str_preds = generate_predictions(model_en, tokenizer_en, peng_tok["lap14"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oas") for el in str_preds]

11it [00:06,  1.71it/s]


In [44]:
labels = [catch_answer(el,"oas") for el in peng_2["lap14"]["test"]["output"]]

In [45]:
summary_score(preds,labels)

{'recall': 0.5027726432532348,
 'precision': 0.6570048309178744,
 'f1_score': 0.5696335078534032}

In [46]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [47]:
from numba import cuda
numba_device = cuda.get_current_device()
numba_device.reset()

In [52]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

# Peng Restaurant 2014

In [54]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["peng"]["res14"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["res14"]["train"],
        eval_dataset = peng_tok["res14"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["res14"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["peng"]["res14"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, peng_tok["res14"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oas") for el in str_preds]

16it [00:17,  1.11s/it]


In [ ]:
labels = [catch_answer(el,"oas") for el in peng_2["res14"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.4305835010060362,
 'precision': 0.7210084033613445,
 'f1_score': 0.5391742012021976}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# Peng Restaurant 2015

In [ ]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["peng"]["res15"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["res15"]["train"],
        eval_dataset = peng_tok["res15"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["res15"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["peng"]["res15"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

Epoch,Training Loss,Validation Loss


TrainOutput(global_step=1520, training_loss=0.03920878225104197, metrics={'train_runtime': 763.4677, 'train_samples_per_second': 63.395, 'train_steps_per_second': 1.991, 'total_flos': 7138128273408000.0, 'train_loss': 0.03920878225104197, 'epoch': 20.0})

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, peng_tok["res15"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oas") for el in str_preds]

11it [00:09,  1.11it/s]


In [ ]:
labels = [catch_answer(el,"oas") for el in peng_2["res15"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.5298969072164949,
 'precision': 0.6277372262773723,
 'f1_score': 0.5746824581702831}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# Peng Restaurant 2016

In [ ]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["peng"]["res16"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = peng_tok["res16"]["train"],
        eval_dataset = peng_tok["res16"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,peng_2["res16"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["peng"]["res16"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

Epoch,Training Loss,Validation Loss,Overall Recall,Overall Precision,Overall F1 Score,Oas Recall,Oas Precision,Oas F1 Score
1,0.504400,0.088252,0.575221,0.655518,0.612750,0.575221,0.655518,0.612750
2,0.032200,0.083801,0.613569,0.726644,0.665336,0.613569,0.726644,0.665336


***** Running Evaluation *****
  Num examples = 210
  Num examples = 210
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-108
Configuration saved in ./t5/checkpoint-108/config.json
Saving model checkpoint to ./t5/checkpoint-108
Configuration saved in ./t5/checkpoint-108/config.json
Model weights saved in ./t5/checkpoint-108/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-108/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-108/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-108/spiece.model
***** Running Evaluation *****
  Num examples = 210
  Batch size = 32
***** Running Evaluation *****
  Num examples = 210
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-216
Configuration saved in ./t5/checkpoint-216/config.json
Saving model checkpoint to ./t5/checkpoint-216
Configuration saved in ./t5/checkpoint-216/config.json
Model weights saved in ./t5/checkpoint-216/pytorch_model.bin
tokenizer config file saved in ./t5/check

TrainOutput(global_step=2160, training_loss=0.030442173609992972, metrics={'train_runtime': 1022.3089, 'train_samples_per_second': 67.064, 'train_steps_per_second': 2.113, 'total_flos': 1.011127478188032e+16, 'train_loss': 0.030442173609992972, 'epoch': 20.0})

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, peng_tok["res16"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oas") for el in str_preds]

11it [00:11,  1.05s/it]


In [ ]:
labels = [catch_answer(el,"oas") for el in peng_2["res16"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.603112840466926,
 'precision': 0.6813186813186813,
 'f1_score': 0.6398348813209495}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# Wan Restaurant 2015

In [ ]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["wan"]["res15"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = wan_tok["res15"]["train"],
        eval_dataset = wan_tok["res15"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,wan_2["res15"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["wan"]["res15"].train()

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5600
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3500
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 10
  Batch size = 32
  Num examples = 10
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-175
Saving model checkpoint to ./t5/checkpoint-175
Configuration saved in ./t5/checkpoint-175/config.json
Model weights saved in ./t5/checkpoint-175/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-175/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-175/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-175/spiece.model
***** Running Evaluation *****
  Num examples = 10
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-350
Configuration saved in ./t5/checkpoint-350/config.json
Model weights saved in ./t5/checkpoint-350/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-350/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-350/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-350/spiece.model
Saving model checkpoint to ./t5/checkpoint-525
Configur

TrainOutput(global_step=3500, training_loss=0.021271900387746945, metrics={'train_runtime': 1659.1202, 'train_samples_per_second': 67.506, 'train_steps_per_second': 2.11, 'total_flos': 1.79832877056e+16, 'train_loss': 0.021271900387746945, 'epoch': 20.0})

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, wan_tok["res15"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"asc") for el in str_preds]

19it [00:10,  1.83it/s]


In [ ]:
labels = [catch_answer(el,"asc") for el in wan_2["res15"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.4165680473372781,
 'precision': 0.6037735849056604,
 'f1_score': 0.49299719887955185}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# Wan Restaurant 2016

In [ ]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["wan"]["res16"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = wan_tok["res16"]["train"],
        eval_dataset = wan_tok["res16"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,wan_2["res16"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["wan"]["res16"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

: 

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, wan_tok["res16"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"asc") for el in str_preds]

19it [00:10,  1.74it/s]


In [ ]:
labels = [catch_answer(el,"asc") for el in wan_2["res16"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.027939464493597205,
 'precision': 0.04411764705882353,
 'f1_score': 0.03421240199572345}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# Zhang Restaurant 2015

In [ ]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["zhang"]["res15"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = zhang_tok["res15"]["train"],
        eval_dataset = zhang_tok["res15"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,zhang_2["res15"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["zhang"]["res15"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.75 GiB total capacity; 28.43 GiB already allocated; 21.94 MiB free; 29.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, zhang_tok["res15"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oasc") for el in str_preds]

16it [00:13,  1.19it/s]


In [ ]:
labels = [catch_answer(el,"oasc") for el in zhang_2["res15"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.4305835010060362,
 'precision': 0.7210084033613445,
 'f1_score': 0.5391742012021976}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# Zhang Restaurant 2016

In [ ]:
model_en = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
model_en.to(device)
trainer["zhang"]["res16"] = Seq2SeqTrainer(
        model = model_en,
        args = train_args,
        tokenizer = tokenizer_en,
        data_collator = data_collator_en,
        train_dataset = zhang_tok["res16"]["train"],
        eval_dataset = zhang_tok["res16"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,zhang_2["res16"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["zhang"]["res16"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 310
  Batch size = 32
  Num examples = 310
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-159
Configuration saved in ./t5/checkpoint-159/config.json
Model weights saved in ./t5/checkpoint-159/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-159/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-159/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-159/spiece.model
***** Running Evaluation *****
  Num examples = 310
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-318
Configuration saved in ./t5/checkpoint-318/config.json
Model weights saved in ./t5/checkpoint-318/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-318/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-318/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-318/spiece.model
***** Running Evaluation *****
  Num examples = 310
  Batch size = 32
Saving model checkpoint to ./

TrainOutput(global_step=3180, training_loss=0.02443723297156628, metrics={'train_runtime': 1613.4167, 'train_samples_per_second': 62.774, 'train_steps_per_second': 1.971, 'total_flos': 1.397309066698752e+16, 'train_loss': 0.02443723297156628, 'epoch': 20.0})

In [ ]:
str_preds = generate_predictions(model_en, tokenizer_en, zhang_tok["res16"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oasc") for el in str_preds]

16it [00:13,  1.19it/s]


In [ ]:
labels = [catch_answer(el,"oasc") for el in zhang_2["res16"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.4305835010060362,
 'precision': 0.7210084033613445,
 'f1_score': 0.5391742012021976}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()

# William Hotel

In [ ]:
model_id = AutoModelForSeq2SeqLM.from_pretrained("Wikidepia/IndoT5-base")
model_en.to(device)
trainer["william"]["hotel"] = Seq2SeqTrainer(
        model = model_id,
        args = train_args,
        tokenizer = tokenizer_id,
        data_collator = data_collator_en,
        train_dataset = william_tok["hotel"]["train"],
        eval_dataset = william_tok["hotel"]["val"],
        compute_metrics = lambda eval_preds: compute_metrics(eval_preds,decoding_args,tokenizer_en,william_2["hotel"]["val"]["task"]),
        preprocess_logits_for_metrics = preprocess_logits_for_metrics
    )

trainer["william"]["hotel"].train()

loading configuration file config.json from cache at /home/m13519061/.cache/huggingface/hub/models--t5-base/snapshots/fe6d9bf207cd3337512ca838a8b453f87a9178ef/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 310
  Batch size = 32
  Num examples = 310
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-159
Configuration saved in ./t5/checkpoint-159/config.json
Model weights saved in ./t5/checkpoint-159/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-159/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-159/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-159/spiece.model
***** Running Evaluation *****
  Num examples = 310
  Batch size = 32
Saving model checkpoint to ./t5/checkpoint-318
Configuration saved in ./t5/checkpoint-318/config.json
Model weights saved in ./t5/checkpoint-318/pytorch_model.bin
tokenizer config file saved in ./t5/checkpoint-318/tokenizer_config.json
Special tokens file saved in ./t5/checkpoint-318/special_tokens_map.json
Copy vocab file to ./t5/checkpoint-318/spiece.model
***** Running Evaluation *****
  Num examples = 310
  Batch size = 32
Saving model checkpoint to ./

TrainOutput(global_step=3180, training_loss=0.02443723297156628, metrics={'train_runtime': 1613.4167, 'train_samples_per_second': 62.774, 'train_steps_per_second': 1.971, 'total_flos': 1.397309066698752e+16, 'train_loss': 0.02443723297156628, 'epoch': 20.0})

In [ ]:
str_preds = generate_predictions(model_id, tokenizer_id, william_tok["hotel"]["test"], device, 32, 512, decoding_args)
preds = [catch_answer(el,"oas") for el in str_preds]

16it [00:13,  1.19it/s]


In [ ]:
labels = [catch_answer(el,"oas") for el in william_2["hotel"]["test"]["output"]]

In [ ]:
summary_score(preds,labels)

{'recall': 0.4305835010060362,
 'precision': 0.7210084033613445,
 'f1_score': 0.5391742012021976}

In [ ]:
!rm -rf ./t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
numba_device = cuda.get_current_device()
numba_device.reset()